In [1]:
import tensorflow as tf
import os
import glob
import tarfile
import numpy as np
from scipy.io import loadmat
from shutil import copyfile, rmtree
import sys
import json
if sys.version_info[0] >= 3:
    from urllib.request import urlretrieve
else:
    from urllib import urlretrieve

In [2]:
def download_file(url, dest=None):
    if not dest:
        dest = os.path.join(data_path, url.split('/')[-1])
    urlretrieve(url, dest)

In [3]:
data_path = 'flower'

In [4]:
if not os.path.exists(data_path):
    os.mkdir(data_path)
flowers_archive_path = os.path.join(data_path, '102flowers.tgz')
if not os.path.isfile(flowers_archive_path):
    print ('Downloading images...')
    download_file('http://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz')
    tarfile.open(flowers_archive_path).extractall(path=data_path)

image_labels_path = os.path.join(data_path, 'imagelabels.mat')
if not os.path.isfile(image_labels_path):
    print("Downloading image labels...")
    download_file('http://www.robots.ox.ac.uk/~vgg/data/flowers/102/imagelabels.mat')

In [5]:
image_labels = loadmat(image_labels_path)['labels'][0]
image_labels -= 1

In [6]:
files = sorted(glob.glob(os.path.join(data_path, 'jpg', '*.jpg')))
labels = np.array([i for i in zip(files, image_labels)])

In [7]:
cwd = os.getcwd()

In [8]:
def move_files(dir_name,cwd,labels):
    cur_dir_path = os.path.join(cwd, dir_name)
    if not os.path.exists(cur_dir_path):
        os.mkdir(cur_dir_path)
    for i in range(0, 102):
        class_dir = os.path.join(cwd, dir_name, str(i))
        os.mkdir(class_dir)
    for label in labels:
        src = str(label[0])
        dst = os.path.join(cwd,dir_name, label[1], src.split(os.sep)[-1])
        copyfile(src, dst)

In [9]:
dir_name=os.path.join(data_path,'class')
move_files(dir_name,cwd,labels)

In [10]:
def save_dict(content,filename):
    content = dict(content)
    with open(filename,'w') as file_object:
        json.dump(content,file_object)

In [11]:
def load_dict(filename):
    with open(filename,'r') as file_object:
        content = json.load(file_object)
    return content

In [12]:
save_dict(labels,os.path.join(data_path,'image-label.json'))

In [13]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Set the seed for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

# Dataset directory
dataset_directory = '/content/flower/class'

# Split ratio
train_ratio = 0.85

# Load and preprocess the images
image_size = (224, 224)
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    validation_split=1 - train_ratio,
)

train_generator = datagen.flow_from_directory(
    dataset_directory,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
)

validation_generator = datagen.flow_from_directory(
    dataset_directory,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
)

# Load pre-trained ResNet-50 model without top classification layers
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(image_size[0], image_size[1], 3),
)

# Add custom classification layers
x = GlobalAveragePooling2D()(base_model.output)
output = Dense(train_generator.num_classes, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'],
)

# Train the model
epochs = 10

model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
)

# Save the model (optional)
model.save('/content/')

# Evaluate the model on a test dataset (optional)
#test_generator = datagen.flow_from_directory(
#    '/path/to/test_dataset',
#    target_size=image_size


Found 7004 images belonging to 102 classes.
Found 1185 images belonging to 102 classes.
94765736/94765736 [==============================] - 5s 0us/step
Epoch 1/10
219/219 [==============================] - 144s 428ms/step - loss: 1.8609 - accuracy: 0.5557 - val_loss: 98.2551 - val_accuracy: 0.0076
Epoch 2/10
219/219 [==============================] - 90s 410ms/step - loss: 0.5211 - accuracy: 0.8475 - val_loss: 12.4189 - val_accuracy: 0.0068
Epoch 3/10
219/219 [==============================] - 90s 411ms/step - loss: 0.2527 - accuracy: 0.9266 - val_loss: 8.1714 - val_accuracy: 0.0262
Epoch 4/10
219/219 [==============================] - 89s 407ms/step - loss: 0.2100 - accuracy: 0.9376 - val_loss: 3.4475 - val_accuracy: 0.3958
Epoch 5/10
219/219 [==============================] - 90s 408ms/step - loss: 0.1199 - accuracy: 0.9674 - val_loss: 1.2754 - val_accuracy: 0.6979
Epoch 6/10
219/219 [==============================] - 89s 406ms/step - loss: 0.1174 - accuracy: 0.9669 - val_loss: 1.09

In [14]:
model.save(os.path.join('model','model.h5'))

In [15]:
class_labels = [
    'pink primrose',
    'hard-leaved pocket orchid',
    'canterbury bells',
    'sweet pea',
    'english marigold',
    'tiger lily',
    'moon orchid',
    'bird of paradise',
    'monkshood',
    'globe thistle',
    'snapdragon',
    "colt's foot",
    'king protea',
    'spear thistle',
    'yellow iris',
    'globe-flower',
    'purple coneflower',
    'peruvian lily',
    'balloon flower',
    'giant white arum lily',
    'fire lily',
    'pincushion flower',
    'fritillary',
    'red ginger',
    'grape hyacinth',
    'corn poppy',
    'prince of wales feathers',
    'stemless gentian',
    'artichoke',
    'sweet william',
    'carnation',
    'garden phlox',
    'love in the mist',
    'mexican aster',
    'alpine sea holly',
    'ruby-lipped cattleya',
    'cape flower',
    'great masterwort',
    'siam tulip',
    'lenten rose',
    'barbeton daisy',
    'daffodil',
    'sword lily',
    'poinsettia',
    'bolero deep blue',
    'wallflower',
    'marigold',
    'buttercup',
    'oxeye daisy',
    'common dandelion',
    'petunia',
    'wild pansy',
    'primula',
    'sunflower',
    'pelargonium',
    'bishop of llandaff',
    'gaura',
    'geranium',
    'orange dahlia',
    'pink-yellow dahlia?',
    'cautleya spicata',
    'japanese anemone',
    'black-eyed susan',
    'silverbush',
    'californian poppy',
    'osteospermum',
    'spring crocus',
    'bearded iris',
    'windflower',
    'tree poppy',
    'gazania',
    'azalea',
    'water lily',
    'rose',
    'thorn apple',
    'morning glory',
    'passion flower',
    'lotus',
    'toad lily',
    'anthurium',
    'frangipani',
    'clematis',
    'hibiscus',
    'columbine',
    'desert-rose',
    'tree mallow',
    'magnolia',
    'cyclamen ',
    'watercress',
    'canna lily',
    'hippeastrum ',
    'bee balm',
    'ball moss',
    'foxglove',
    'bougainvillea',
    'camellia',
    'mallow',
    'mexican petunia',
    'bromelia',
    'blanket flower',
    'trumpet creeper',
    'blackberry lily'
]

In [17]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('/content/model/model.h5')

# Directory containing the images for prediction
images_directory = '/content/flower/class/1'

# List the images in the directory
image_files = os.listdir(images_directory)

# Make predictions for each image
for image_file in image_files:
    # Load and preprocess the image
    img_path = os.path.join(images_directory, image_file)
    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img /= 255.0  # Normalize the image

    # Make prediction
    predictions = model.predict(img)
    predicted_class_index = np.argmax(predictions[0])

    # Get the predicted class label
    # Assuming you have a list of class labels used during training
     # Update with your own labels
    predicted_class_label = class_labels[predicted_class_index]

    # Print the predicted class label
    print(f"Image: {image_file} | Predicted class: {predicted_class_label}")


1/1 [==============================] - 1s 1s/step
Image: image_05107.jpg | Predicted class: hard-leaved pocket orchid
1/1 [==============================] - 0s 26ms/step
Image: image_05134.jpg | Predicted class: hard-leaved pocket orchid
1/1 [==============================] - 0s 28ms/step
Image: image_05145.jpg | Predicted class: hard-leaved pocket orchid
1/1 [==============================] - 0s 33ms/step
Image: image_05130.jpg | Predicted class: hard-leaved pocket orchid
1/1 [==============================] - 0s 29ms/step
Image: image_05091.jpg | Predicted class: hard-leaved pocket orchid
1/1 [==============================] - 0s 31ms/step
Image: image_05137.jpg | Predicted class: ruby-lipped cattleya
1/1 [==============================] - 0s 30ms/step
Image: image_05146.jpg | Predicted class: hard-leaved pocket orchid
1/1 [==============================] - 0s 26ms/step
Image: image_05104.jpg | Predicted class: hard-leaved pocket orchid
1/1 [==============================] - 0s 26ms/